In [ ]:
import os
import sys
import pathlib
import subprocess
import glob
import pandas as pd
import numpy as np
import xarray as xr
import dask.array as da
import time

# define filepaths
root = pathlib.Path("..").resolve()
chrtout_path = os.path.join(root,"test","input","geo","NWM_2.1_Sample_Datasets","Pocono_TEST1","example_CHRTOUT")

chrtout_files = glob.glob(os.path.join(chrtout_path,"*DOMAIN1"))

# list of paths to which to save edited CHRTOUT files
chrtout_files_new = []
chrtout_files_new[:] = [s + ".TRTE" for s in chrtout_files]


In [ ]:
def drop_all_coords(ds):
    return ds.reset_coords(drop=True)

# open a bunch of CHRTOUT files
stack = xr.open_mfdataset(
        chrtout_files,
        combine="by_coords",
#         preprocess=drop_all_coords,
    )

# Hypothetical Q data from t-route, formatted as 3d numpy
# index organization?
qTroute = np.random.randn(len(stack.time),len(stack.feature_id)).astype("float32")

# convert numpy array to dask array 
qTroute_dask = da.from_array(qTroute, chunks = (1,374))

# package flow data (as dask array) into xarray DataArray
# dims and coords match exactly those in chrtout
new_data = xr.DataArray(
    data=qTroute_dask,
    dims=["time","feature_id"],
    coords=dict(
        time = stack.time.values,
        feature_id=stack.feature_id.values
    ),
    attrs=dict(
        description="t-route streamflow",
        units="cubic meters per second",
    ),
)

# add t-route flow variable to CHRTOUT stack
stack["qTroute"] = new_data

# build a list of datasets, 1 for each timestep
ts = time.time()
dataset_list = []
grp_object = list(stack.groupby("time"))
for i in range(len(list(stack.groupby("time")))):
    dataset_list.append(grp_object[i][1])
    dataset_list[i] = dataset_list[i].expand_dims({"time": [grp_object[i][0]]})
tf = time.time()
print("splitting out datasets took:",tf - ts,"seconds")

# write edited chrtoutfiles to disk
ts = time.time()
xr.save_mfdataset(dataset_list, paths = chrtout_files_new)
tf = time.time()
print("saving netcdfs to disk took:", tf  - ts, "seconds")